In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import cv2
import sklearn
from tensorflow.keras import regularizers
from tensorflow.keras import layers
import os

In [2]:
GESTURE_TYPES = 11
LABEL_DICT = {k:i for i,k in enumerate([21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])}
CONNECTION_LABELS = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

In [3]:
def load_keypoint_sequences(data_path='gesture_recognition/Fall 2020/data_30'):
    keypoints = []
    labels = []
    for subjectName in os.listdir(data_path):
        if not (subjectName.startswith("Subject") or subjectName.startswith("subject")): continue
        # subjectNum = int(re.findall(r'(\d+)', subjectName)[0])
        for sceneName in os.listdir(os.path.join(data_path, subjectName)):
            if not (sceneName.startswith("Scene") or subjectName.startswith("scene")): continue
            for groupEntry in os.scandir(os.path.join(data_path, subjectName, sceneName)):
                with open(groupEntry, 'r') as f:
                    groupData = json.load(f)
                    for gesture in groupData:
                        # print(gesture['label'], gesture['keypoints'])
                        for i in range(len(gesture['keypoints'])):
                            if not gesture['keypoints'][i]:
                                gesture['keypoints'][i] = [[np.nan, np.nan, np.nan] for _ in range(21)]
                        keypoints.append(gesture['keypoints'])
                        labels.append(LABEL_DICT[gesture['label']])
    keypoints = tf.keras.preprocessing.sequence.pad_sequences(keypoints,dtype='float32',padding='post',value=np.NaN)
    labels = np.array(labels)
    return keypoints, labels
keypoints, labels = load_keypoint_sequences()
print(keypoints.shape, labels.shape)

(3141, 109, 21, 3) (3141,)


In [4]:
def generate_connection_angles_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    angles = (tensor1*tensor2).sum(axis=-1)/np.linalg.norm(tensor1,axis=-1)/np.linalg.norm(tensor2,axis=-1)
    angles = angles.transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return np.arccos(angles)
angles = generate_connection_angles_from_sequences(keypoints)
print(angles.shape)

(3141, 109, 210)


In [5]:
def generate_joint_distances_from_sequences(keypoints, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    distances = np.linalg.norm(tensor1-tensor2,axis=-1).transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return distances
distances = generate_joint_distances_from_sequences(keypoints)
print(distances.shape)

(3141, 109, 210)


In [6]:
def visualize_keypoint_sequences(keypoints):
    for sequence in keypoints:
        for points in sequence:
            img = np.zeros((480, 640, 3))
            for point in points:
                x, y, z = point
                if np.isnan(x):
                    continue
                cv2.circle(img, (int(x), int(y)), 4, (255, 0, 0), 2)
            for connection in CONNECTION_LABELS:
                if np.isnan(points[connection[0]][0]):
                    continue
                x0, y0, z0 = points[connection[0]]
                x1, y1, z1 = points[connection[1]]
                cv2.line(img, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
            cv2.imshow("Key Points", img)
            key = cv2.waitKey(1)
            if key == 27:
                cv2.destroyAllWindows()
                cv2.waitKey(1) # cannot close window on macOS without this line
                return
# visualize_keypoint_sequences(keypoints)

In [7]:
def process_sequence_features(keypoints, angles, distances):
    data_length = keypoints.shape[0]
    sequence_length = keypoints.shape[1]
    keypoints = keypoints.reshape(data_length*sequence_length, -1)
    angles = angles.reshape(data_length*sequence_length, -1)
    distances = distances.reshape(data_length*sequence_length, -1)
    features = np.concatenate((keypoints, angles, distances), -1)
    df = pd.DataFrame(features)
    df = (df-df.mean())/df.std()
    df = df.fillna(0)
    features = df.to_numpy().reshape(data_length, sequence_length, -1)
    return features
X = process_sequence_features(keypoints, angles, distances)
#normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
#normalizer.adapt(X)
# X_train, X_val, y_train, y_val = train_test_split(processed_keypoints, labels, test_size=0.2, random_state=0)
print(X.shape)

(3141, 109, 483)


In [8]:
sequence_length = X.shape[1]
sequence_labels = np.tile(np.expand_dims(labels,(1,2)),[1,sequence_length,1])
print(sequence_labels.shape)

(3141, 109, 1)


## RNN Classification

In [10]:
model_lstm = tf.keras.Sequential([layers.Masking(), layers.LSTM(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_lstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm.fit(X, labels, epochs=30, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/30
2512/2512 [==============================] - 20s 8ms/sample - loss: 5049.8210 - accuracy: 0.1162 - val_loss: 4805.8862 - val_accuracy: 0.1081
Epoch 2/30
2512/2512 [==============================] - 16s 6ms/sample - loss: 7702.2804 - accuracy: 0.1342 - val_loss: 291.5189 - val_accuracy: 0.1558
Epoch 3/30
2512/2512 [==============================] - 16s 6ms/sample - loss: 141.5456 - accuracy: 0.1680 - val_loss: 270.7291 - val_accuracy: 0.1399
Epoch 4/30
2512/2512 [==============================] - 17s 7ms/sample - loss: 283.5316 - accuracy: 0.1015 - val_loss: 520.3309 - val_accuracy: 0.0588
Epoch 5/30
2512/2512 [==============================] - 17s 7ms/sample - loss: 330.9073 - accuracy: 0.0593 - val_loss: 480.6600 - val_accuracy: 0.0525
Epoch 6/30
2512/2512 [==============================] - 17s 7ms/sample - loss: 303.5358 - accuracy: 0.0561 - val_loss: 424.4876 - val_accuracy: 0.0620
Epoch 7/30
2512/2512 [======================

LSTM performs surprisingly poorly on super long sequence.

In [9]:
model_gru = tf.keras.Sequential([layers.Masking(), layers.GRU(GESTURE_TYPES, activation=None), layers.Activation('softmax')])
model_gru.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_gru.fit(X, labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 23s 9ms/sample - loss: 1.8083 - accuracy: 0.3909 - val_loss: 1.1103 - val_accuracy: 0.6502
Epoch 2/20
2512/2512 [==============================] - 19s 8ms/sample - loss: 0.7383 - accuracy: 0.7715 - val_loss: 0.7026 - val_accuracy: 0.7838
Epoch 3/20
2512/2512 [==============================] - 19s 8ms/sample - loss: 0.4758 - accuracy: 0.8583 - val_loss: 0.6364 - val_accuracy: 0.7949
Epoch 4/20
2512/2512 [==============================] - 21s 8ms/sample - loss: 0.3836 - accuracy: 0.8826 - val_loss: 0.5950 - val_accuracy: 0.8188
Epoch 5/20
2512/2512 [==============================] - 19s 8ms/sample - loss: 0.3318 - accuracy: 0.8993 - val_loss: 0.5756 - val_accuracy: 0.8315
Epoch 6/20
2512/2512 [==============================] - 19s 8ms/sample - loss: 0.3001 - accuracy: 0.9092 - val_loss: 0.5744 - val_accuracy: 0.8203
Epoch 7/20
2512/2512 [==============================] - 19s 7ms/sample 

In [10]:
model_bilstm = tf.keras.Sequential([layers.Masking(), tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(GESTURE_TYPES, activation=None)), tf.keras.layers.Activation('softmax')])
model_bilstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_bilstm.fit(X, labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 45s 18ms/sample - loss: 27.0727 - accuracy: 0.0748 - val_loss: 31.6009 - val_accuracy: 0.0986
Epoch 2/20
2512/2512 [==============================] - 38s 15ms/sample - loss: 42.2187 - accuracy: 0.1576 - val_loss: 51.9413 - val_accuracy: 0.1558
Epoch 3/20
2512/2512 [==============================] - 38s 15ms/sample - loss: 52.4383 - accuracy: 0.1278 - val_loss: 37.4875 - val_accuracy: 0.1367
Epoch 4/20
2512/2512 [==============================] - 38s 15ms/sample - loss: 63.5021 - accuracy: 0.0828 - val_loss: 221.7672 - val_accuracy: 0.0604
Epoch 5/20
2512/2512 [==============================] - 39s 16ms/sample - loss: 107.0027 - accuracy: 0.0657 - val_loss: 205.7569 - val_accuracy: 0.0715
Epoch 6/20
2512/2512 [==============================] - 38s 15ms/sample - loss: 95.4847 - accuracy: 0.0665 - val_loss: 203.5869 - val_accuracy: 0.0827
Epoch 7/20
2512/2512 [===========================

Making LSTM bidirectional damages the performance when sequence is very long.

In [11]:
model_rnn = tf.keras.Sequential([layers.Masking() ,tf.keras.layers.SimpleRNN(GESTURE_TYPES, activation=None), tf.keras.layers.Activation('softmax')])
model_rnn.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_rnn.fit(X, labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 14s 6ms/sample - loss: 3.7375 - accuracy: 0.1672 - val_loss: 2.6367 - val_accuracy: 0.2130
Epoch 2/20
2512/2512 [==============================] - 13s 5ms/sample - loss: 2.2993 - accuracy: 0.2528 - val_loss: 2.3028 - val_accuracy: 0.2655
Epoch 3/20
2512/2512 [==============================] - 14s 5ms/sample - loss: 2.0297 - accuracy: 0.3153 - val_loss: 2.1143 - val_accuracy: 0.2893
Epoch 4/20
2512/2512 [==============================] - 13s 5ms/sample - loss: 1.8215 - accuracy: 0.3718 - val_loss: 1.9863 - val_accuracy: 0.3291
Epoch 5/20
2512/2512 [==============================] - 13s 5ms/sample - loss: 1.6846 - accuracy: 0.4092 - val_loss: 1.8590 - val_accuracy: 0.3609
Epoch 6/20
2512/2512 [==============================] - 12s 5ms/sample - loss: 1.5658 - accuracy: 0.4562 - val_loss: 1.8152 - val_accuracy: 0.3927
Epoch 7/20
2512/2512 [==============================] - 13s 5ms/sample 

Simple RNN gives a slightly worse performance.

In [12]:
model_lstm4 = tf.keras.Sequential([layers.Masking() ,tf.keras.layers.LSTM(128), layers.Dense(GESTURE_TYPES), layers.Activation('softmax')])
model_lstm4.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm4.fit(X, labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 46s 18ms/sample - loss: 0.8413 - accuracy: 0.7504 - val_loss: 0.6189 - val_accuracy: 0.8188
Epoch 2/20
2512/2512 [==============================] - 38s 15ms/sample - loss: 0.3777 - accuracy: 0.8881 - val_loss: 0.5334 - val_accuracy: 0.8315
Epoch 3/20
2512/2512 [==============================] - 37s 15ms/sample - loss: 0.2432 - accuracy: 0.9327 - val_loss: 0.5423 - val_accuracy: 0.8347
Epoch 4/20
2512/2512 [==============================] - 37s 15ms/sample - loss: 0.1909 - accuracy: 0.9471 - val_loss: 0.5253 - val_accuracy: 0.8378
Epoch 5/20
2512/2512 [==============================] - 37s 15ms/sample - loss: 0.1330 - accuracy: 0.9650 - val_loss: 0.5498 - val_accuracy: 0.8347
Epoch 6/20
2512/2512 [==============================] - 36s 15ms/sample - loss: 0.0914 - accuracy: 0.9825 - val_loss: 0.5445 - val_accuracy: 0.8474
Epoch 7/20
2512/2512 [==============================] - 37s 15ms/

Increasing depth may improve performance.

Sequential output (many to many)

In [11]:

model_lstm_sequence = tf.keras.Sequential([layers.Masking(), layers.LSTM(128, return_sequences=True), layers.TimeDistributed(layers.Dense(GESTURE_TYPES, activation='softmax'))])
model_lstm_sequence.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_lstm_sequence.fit(X, sequence_labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 38s 15ms/sample - loss: 0.2688 - accuracy: 0.7058 - val_loss: 0.2200 - val_accuracy: 0.7478
Epoch 2/20
2512/2512 [==============================] - 30s 12ms/sample - loss: 0.1770 - accuracy: 0.8030 - val_loss: 0.1991 - val_accuracy: 0.7646
Epoch 3/20
2512/2512 [==============================] - 31s 12ms/sample - loss: 0.1530 - accuracy: 0.8292 - val_loss: 0.2017 - val_accuracy: 0.7675
Epoch 4/20
2512/2512 [==============================] - 31s 12ms/sample - loss: 0.1409 - accuracy: 0.8398 - val_loss: 0.1891 - val_accuracy: 0.7736
Epoch 5/20
2512/2512 [==============================] - 31s 12ms/sample - loss: 0.1279 - accuracy: 0.8552 - val_loss: 0.1905 - val_accuracy: 0.7808
Epoch 6/20
2512/2512 [==============================] - 30s 12ms/sample - loss: 0.1120 - accuracy: 0.8752 - val_loss: 0.1923 - val_accuracy: 0.7774
Epoch 7/20
2512/2512 [==============================] - 31s 12ms/

In [12]:
model_gru_sequence = tf.keras.Sequential([layers.Masking(), layers.GRU(128, return_sequences=True), layers.TimeDistributed(layers.Dense(GESTURE_TYPES, activation='softmax'))])
model_gru_sequence.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics=['accuracy'])
history = model_gru_sequence.fit(X, sequence_labels, epochs=20, validation_split=0.2)

Train on 2512 samples, validate on 629 samples
Epoch 1/20
2512/2512 [==============================] - 33s 13ms/sample - loss: 0.2756 - accuracy: 0.6868 - val_loss: 0.2295 - val_accuracy: 0.7389
Epoch 2/20
2512/2512 [==============================] - 26s 10ms/sample - loss: 0.1828 - accuracy: 0.7924 - val_loss: 0.2082 - val_accuracy: 0.7550
Epoch 3/20
2512/2512 [==============================] - 27s 11ms/sample - loss: 0.1548 - accuracy: 0.8249 - val_loss: 0.1895 - val_accuracy: 0.7749
Epoch 4/20
2512/2512 [==============================] - 27s 11ms/sample - loss: 0.1358 - accuracy: 0.8460 - val_loss: 0.1932 - val_accuracy: 0.7730
Epoch 5/20
2512/2512 [==============================] - 27s 11ms/sample - loss: 0.1205 - accuracy: 0.8660 - val_loss: 0.1918 - val_accuracy: 0.7749
Epoch 6/20
2512/2512 [==============================] - 27s 11ms/sample - loss: 0.1068 - accuracy: 0.8811 - val_loss: 0.1871 - val_accuracy: 0.7824
Epoch 7/20
2512/2512 [==============================] - 27s 11ms/

In general, GRU is a more stable and cheaper solution compared to LSTM for our problem setting.